In [1]:
# ==========================
# FULL WORKING PROJECT CODE
# ==========================

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

# --------------------------
# 1. DATA GENERATION
# --------------------------
np.random.seed(42)
t = np.arange(1095)
series1 = 0.05*t + 15*np.sin(2*np.pi*t/365) + np.random.normal(0,2,1095)
series2 = 0.7*series1 + np.random.normal(0,1,1095)
series3 = np.cumsum(np.random.normal(0,1,1095))

df = pd.DataFrame({
    'Sales_Target': series1,
    'Foot_Traffic': series2,
    'Market_Noise': series3
})

# --------------------------
# 2. SCALING
# --------------------------
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# --------------------------
# 3. CREATE SEQUENCES
# --------------------------
def create_sequences(data, seq_length=7):
    X, y = [], []
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length,0])
    return np.array(X), np.array(y)

X, y = create_sequences(data)

split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32).view(-1,1)
y_test = torch.tensor(y_test,dtype=torch.float32).view(-1,1)

# --------------------------
# 4. LSTM MODEL
# --------------------------
class LSTMModel(nn.Module):
    def __init__(self,input_dim=3,hidden_dim=64):
        super().__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,batch_first=True)
        self.fc = nn.Linear(hidden_dim,1)
    def forward(self,x):
        out,_ = self.lstm(x)
        return self.fc(out[:,-1,:])

model = LSTMModel()
optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.MSELoss()

# TRAIN
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    pred = model(X_train)
    loss = criterion(pred,y_train)
    loss.backward()
    optimizer.step()
    if epoch%10==0:
        print(f"Epoch {epoch} Loss {loss.item():.4f}")

# --------------------------
# 5. LSTM METRICS
# --------------------------
model.eval()
with torch.no_grad():
    preds = model(X_test).numpy()
    true = y_test.numpy()

mae = mean_absolute_error(true,preds)
rmse = np.sqrt(mean_squared_error(true,preds))
mape = np.mean(np.abs((true-preds)/true))*100

print("\nLSTM RESULTS")
print("MAE:",mae)
print("RMSE:",rmse)
print("MAPE:",mape)

# --------------------------
# 6. WALK FORWARD LSTM
# --------------------------
def walk_forward_validation(model,X,y):
    errors=[]
    start=int(len(X)*0.6)
    for i in range(start,len(X)-1):
        X_test=X[i:i+1]
        y_true=y[i]
        with torch.no_grad():
            pred=model(X_test).item()
        errors.append(abs(pred-y_true))
    return np.mean(errors)

wf_error = walk_forward_validation(model,X_test,y_test.numpy())
print("Walk Forward LSTM:",wf_error)

# --------------------------
# 7. XGBOOST
# --------------------------
X_xgb = X.reshape(len(X),-1)
y_xgb = y

X_train_xgb, X_test_xgb = X_xgb[:split], X_xgb[split:]
y_train_xgb, y_test_xgb = y_xgb[:split], y_xgb[split:]

xgb_model = XGBRegressor(n_estimators=200,max_depth=4,learning_rate=0.05)
xgb_model.fit(X_train_xgb,y_train_xgb)

preds_xgb = xgb_model.predict(X_test_xgb)

mae_xgb = mean_absolute_error(y_test_xgb,preds_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test_xgb,preds_xgb))
mape_xgb = np.mean(np.abs((y_test_xgb-preds_xgb)/y_test_xgb))*100

print("\nXGBOOST RESULTS")
print("MAE:",mae_xgb)
print("RMSE:",rmse_xgb)
print("MAPE:",mape_xgb)

# --------------------------
# 8. WALK FORWARD XGB
# --------------------------
def walk_forward_xgb(model,X,y):
    errors=[]
    start=int(len(X)*0.6)
    for i in range(start,len(X)-1):
        pred=model.predict(X[i:i+1])[0]
        errors.append(abs(pred-y[i]))
    return np.mean(errors)

wf_xgb = walk_forward_xgb(xgb_model,X_test_xgb,y_test_xgb)
print("Walk Forward XGB:",wf_xgb)

print("\nFINAL COMPARISON COMPLETE")


Epoch 0 Loss 0.3611
Epoch 10 Loss 0.1752
Epoch 20 Loss 0.0342
Epoch 30 Loss 0.0341
Epoch 40 Loss 0.0255

LSTM RESULTS
MAE: 0.16088875
RMSE: 0.16735767
MAPE: 21.74641042947769
Walk Forward LSTM: 0.18438159

XGBOOST RESULTS
MAE: 0.029392872119595133
RMSE: 0.03673968616608929
MAPE: 4.043549682704524
Walk Forward XGB: 0.03210134447350414

FINAL COMPARISON COMPLETE
